# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


Датафрейм содержит 159292 строк, пропуски в которых отсутствуют. Столбец `Unnamed: 0`, очевидно, является лишним и дублирует интедксы, избавимся от него. Столбец `toxic` содержит только 0 и 1, для оптимизации памяти изменим тип данных на 8-ми разрядный.

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df['toxic'] = df['toxic'].astype('int8')

Bert воспринимает максимум 512 фичей на вход, выберем текст длиной не более 512 символов.

In [ ]:
df['length']=df['text'].str.len()
df = df[df['length']<=512]

In [ ]:
df = df.drop('length', axis=1)

Для ускорения работы возьмем выборку из 1000 комментариев.

In [ ]:
df = df.sample(1000, random_state=99).reset_index(drop=True)

In [ ]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   toxic   1000 non-null   int8  
dtypes: int8(1), object(1)
memory usage: 8.9+ KB


,text,toxic
0,GA Review\n:This review is transcluded from Ta...,0
1,Lns and Logs\nHi. I'm a PhD student doing comp...,0
2,Don't revert my Stoufville edits \n\nYou're an...,1
3,"""\n You have been temporarily blocked from edi...",0
4,When was Tex Beneke's version recorded?,0
5,See: (hovah Strong's 1943 ruin mischief),0
6,u suck!!!! \n\nWUTS YOUR PROBLEM U NERD. I JUS...,1
7,a human being. Understand the weakness that re...,0
8,"""\n I have no objection to adding the picture....",0
9,I suggest you give the same message to Rrius. ...,0


В результате получили набор данных, с которым будем работать далее, состоящий из 126349 строк и двух столбцов. Приведенные выше действия позволили сократить объем памяти, занимаемый данными, практически втрое.

Инициализируем предтренированные токенизатор и модель.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

После чего закодируем наш текст с помощью токенизатора,  приведем длину текстов в корпусе к единой(максимальной из всех имеющихся в наборе) длине.

In [ ]:
tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

# если длина текста меньше максимальной, дозаполним ее до конца длины нулями
attention_mask = np.where(padded != 0, 1, 0)

Сгенерируем эмбеддинги.

In [ ]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

Склеим эмбеддинги, тем самым создав обучающие признаки, и зададим целевой признак

In [ ]:
features = np.concatenate(embeddings).reshape(-1, 1)
target = df['toxic']

Разобьем данные на обучающие и тестовые.

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=99)

Подготовка данных окончена, далее можем перейти к подбору гиперпараметров и обучению моделей.

## Обучение

In [ ]:
# соберем результаты работы моделей в один список
f1_scores = []

### DecisionTree

In [ ]:
tree = DecisionTreeClassifier(random_state=99)

In [ ]:
tree_params = {'max_depth': range(7, 15)}

In [ ]:
gsc_tree = GridSearchCV(tree, tree_params, scoring='f1', n_jobs=-1, cv=5)

In [ ]:
gsc_tree.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=99), n_jobs=-1,
             param_grid={'max_depth': range(7, 15)}, scoring='f1')

In [ ]:
gsc_tree.best_params_

{'max_depth': 7}

In [ ]:
f1_scores.append(gsc_tree.best_score_)

### RandomForest

In [ ]:
forest = RandomForestClassifier(random_state=99)

In [ ]:
forest_params =({'max_depth':range(2, 6),
            'n_estimators':range(100, 200, 20),
             })

In [ ]:
gsc_forest = GridSearchCV(forest, forest_params, scoring='f1', n_jobs=-1, cv=5)

In [ ]:
gsc_forest.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=99), n_jobs=-1,
             param_grid={'max_depth': range(2, 6),
                         'n_estimators': range(100, 200, 20)},
             scoring='f1')

In [ ]:
gsc_forest.best_params_

{'max_depth': 2, 'n_estimators': 100}

In [ ]:
f1_scores.append(gsc_forest.best_score_)

### LogisticRegression

In [ ]:
linear = LogisticRegression(random_state=99)

In [ ]:
linear_params =({'penalty':['l1', 'l2', 'elasticnet'],
            'C':[0.1, 0.5, 1],
            'solver':['lbfgs', 'liblinear', 'newton-cg']
             })

In [ ]:
gsc_linear = GridSearchCV(linear, linear_params, scoring='f1', n_jobs=-1, cv=5)

In [ ]:
gsc_linear.fit(features_train, target_train)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: Fi

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=99), n_jobs=-1,
             param_grid={'C': [0.1, 0.5, 1],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg']},
             scoring='f1')

In [ ]:
gsc_linear.best_params_

{'C': 0.5, 'penalty': 'l2', 'solver': 'lbfgs'}

In [ ]:
f1_scores.append(gsc_linear.best_score_)

In [ ]:
pd.DataFrame(f1_scores, columns=['f1 score'],
             index=['DecisionTree', 'RandomForest', 'LogisticRegression'])

,f1 score
DecisionTree,0.924040
RandomForest,0.930096
LogisticRegression,0.946312


Лучше всего себя показала модель Логистической регрессии со значением метрики равным 0.95. Проверим модель на тестовой выборке.

## Тестирование модели

In [ ]:
predictions = gsc_linear.best_estimator_.predict(features_test.reshape(-1, 1))

In [ ]:
f1_score = f1_score(target_test, predictions)
f1_score

0.9315068493150684

На тестовой выборке метрика показала значение равное 0,93.

## Общий вывод

Загрузили и изучили данные, обнаружили и убрали лишний столбец. Т.к. BERT ограничен 512 фичами, убрали комментарии длиной больше 512 символов. Уровняли баланс классов, даунсемплировав их. Взяли выборку из тысячи комментариев.

Далее выполнили подготовку данных с использованием BERT для обучения и тестирования моделей.

Рассмотрели три модели: дерево решения, случайный лес и логистическую регрессию. Лучше всего себя показала модель логистической регрессии со значением метрики f1 равным 0,95 на обучающей выборке и 0,93 - на тестовой, что можно назвать хорошим результатом.